## DC2: Generate Postage Stamps for set of RA, Dec coordinates

This Notebook demonstrates taking a list of RA, Dec positions and generating "postage-stamp" cutout images from the coadded images.  It illustrates using `matplotlib` for simple display and saving of PNGs.  It also introduces using `lsst.afw.display` to use a more DM-product aware visualization to show the detect object footprints and other mask planes.

### Learning Objectives:
After working through and studying this Notebook you should be able to
   1. Generate a postage stamp from a coadd for your chosen RA, Dec in a chosen filter and save as (a) a FITS file and (b) a PNG.
   2. Use afw.display to show a postage stamp along with the mask planes from the coadd image, including overlaying the object footprints.
   3. [Advanced] Instantiate a Butler object for a repo and read an arbitrary coadd image.  This will require reading through functions carefully and testing individual lines.  
   4. [Advanced] Load the PSF model for a coadd and evaluate it at a given RA, Dec.
   5. [Expert] Look up which tract, patch a given RA, Dec falls in.

### Logistics
This is intended to be runnable at NERSC through the https://jupyter-dev.nersc.gov interface from a local git clone of https://githug.com/LSSTDESC/DC2_Repo in your NERSC directory.  But you can also run it wherever, with appropriate adjustment of the 'repo' location to point to a place where you have a Butler repo will all of the images.

Based in part on https://github.com/lsst-com/notebooks/blob/master/postage_stamp.ipynb

In [ ]:
import numpy as np

from astropy.table import Table

import lsst.daf.persistence as dafPersist
import lsst.afw.geom as afwGeom
import lsst.afw.coord as afwCoord
import lsst.afw.image as afwImage
import lsst.afw.display as afwDisplay

from astropy.visualization import ZScaleInterval

In [ ]:
# Set plotting defaults
%matplotlib notebook
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (8, 8)
zscale = ZScaleInterval()

In [ ]:
def cutout_coadd_ra_dec(butler, ra, dec, filt='r', datasetType='deepCoadd', **kwargs):
    """Produce a cutout from coadd from the given butler at the given RA, Dec in decimal degrees
    
    Trivial wrapper around 'cutout_from_coadd_spherepoint'
    """
    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    return cutout_coadd_spherepoint(butler, radec, filter=filt, datasetType=datasetType)
    
def cutout_coadd_spherepoint(butler, radec, filt='r', datasetType='deepCoadd',
                                  skymap=None, cutoutSideLength=50, **kwargs):
    """Produce a cutout from a coadd at the given afw SpherePoint radec
    

    Parameters
    --
    butler - lsst.daf.persistence.Butler of the data repository
    radec - lsst.afw.geom.SpherePoint coordinates of the center of the cutout.
    filter - Filter of the image to load
    datasetType - 'deepCoadd'  Which type of coadd to load.  Doesn't support 'calexp'
    
    skymap - [optional] Pass in to avoid the Butler read.  Useful if you have lots of them.
    cutoutSideLength - [optional] Side of the cutout region in pixels.
    
    Returns
    --
    MaskedImage
    """
    cutoutSideLength = 50  # pixels
    cutoutSize = afwGeom.ExtentI(cutoutSideLength, cutoutSideLength)

    if skymap is None:
        skymap = butler.get("deepCoadd_skyMap")
    
    tractInfo = skymap.findTract(radec)
    patchInfo = tractInfo.findPatch(radec)
    xy = afwGeom.PointI(tractInfo.getWcs().skyToPixel(radec))
    bbox = afwGeom.BoxI(xy - cutoutSize//2, cutoutSize)

    coaddId = {'tract': tractInfo.getId(), 'patch': "%d,%d" % patchInfo.getIndex(), 'filter': filt}
    
    cutout_image = butler.get(datasetType+'_sub', bbox=bbox, immediate=True, dataId=coaddId)
    
    return cutout_image

In [ ]:
def make_cutout_image(butler, ra, dec, vmin=None, vmax=None, label=None,
                      show=True, saveplot=False, savefits=False,
                      datasetType='deepCoadd'):
    """Generate and optionally display and save a postage stamp for a given RA, Dec.
    
    Returns
    --
    MaskedImage

    Uses matplotlib to generate stamps.  Saves FITS file if requested.
    """

    cutout_image = cutout_coadd_ra_dec(butler, ra, dec, filter='r', datasetType='deepCoadd')
    if savefits:
        if isinstance(savefits, str):
            filename = savefits
        else:
            filename = 'postage-stamp.fits'
        cutout_image.writeFits(filename)
    
    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    xy = cutout_image.getWcs().skyToPixel(radec)
    
    if vmin is None or vmax is None:
        vmin, vmax = zscale.get_limits(cutout_image.image.array)

    plt.imshow(cutout_image.image.array, vmin=vmin, vmax=vmax, cmap='binary')
    plt.colorbar()
    plt.scatter(xy.getX() - cutout_image.getX0(), xy.getY() - cutout_image.getY0(),
                color='none', edgecolor='red', marker='o', s=200)
    if label is not None:
        plt.title(label)
    if saveplot:
        if isinstance(saveplot, str):
            filename = saveplot
        else:
            filename = 'postage-stamp.png'
        plt.savefig(filename)
    if show:
        plt.show()

    return cutout_image

In [ ]:
repo = '/global/projecta/projectdirs/lsst/global/in2p3/Run1.1/output'
butler = dafPersist.Butler(repo)

In [ ]:
filt = 'r'
coord_file = 'id_ra_dec_mid_mag_%s.txt' % filt
id_ra_dec = Table.read(coord_file, format='ascii')

In [ ]:
# Plot just one
first = id_ra_dec[0]
ra, dec = first['RA'], first['DEC']
cutout = make_cutout_image(butler, ra, dec, label="Object ID: %d" % id_ra_dec[0]['ID'])

Because we're using `notebook` as the backend, as long as this is the interactive figure, we can interact with the figure and read off pixel coordinates and counts.

We can loop over this to create postage-stamps for each entry in our catalog.  We turn of `show` here because looping over 20 plots doesn't do very much useful or nice in `notebook` backend.

In [ ]:
show = False
datasetType = 'deepCoadd'
vmin, vmax = -0.75, +1.5  # Fix the vmin, vmax to make it easier to compare across postage stamps.

for objectId, ra, dec in id_ra_dec:
    plt.clf()
    basename = "%s_%s_%s" % (datasetType, objectId, filt)
    saveplot = "%s.png" % basename
    savefits = "%s.fits" % basename
    make_cutout_image(butler, ra, dec, vmin=vmin, vmax=vmax,
                      datasetType=datasetType,
                      label="Object ID: %d" % objectId,
                      show=show, saveplot=saveplot, savefits=savefits)

## AFW Display

But there's additional information available in the Exposure object that we can access using LSST DM tools that are aware of these.  Specifically, we'll use `lsst.afw.display` to expose the mask planes in a cutout image.

In [ ]:
def display_cutout_image(butler, ra, dec, vmin=None, vmax=None, label=None,
                      display=None, backend='matplotlib',
                      show=True, saveplot=False, savefits=False,
                      old_matplotlib = False,
                      datasetType='deepCoadd'):
    """Display a postage stamp for a given RA, Dec using LSST lsst.afw.display.
    
    Returns
    --
    MaskedImage
    
    Backend can be anything that lsst.afw.display and your configuration supports:
       matplotlib, ds9, ginga, firefly.
    You definitely have matplotlib.
    ds9, ginga, and firefly can be set up but are non-trivial on the scale of a simple Notebook
    """
    cutout_image = cutout_coadd_ra_dec(butler, ra, dec, filter='r', datasetType='deepCoadd')
    if savefits:
        if isinstance(savefits, str):
            filename = savefits
        else:
            filename = 'postage-stamp.fits'
        cutout_image.writeFits(filename)
    
    if display is None:
        display = afwDisplay.Display(backend=backend)

    radec = afwGeom.SpherePoint(ra, dec, afwGeom.degrees)
    xy = cutout_image.getWcs().skyToPixel(radec)
    
    display.mtv(cutout_image)
    display.scale("asinh", "zscale")
    display.dot('o', xy.getX(), xy.getY(), ctype='red')
    
    return cutout_image

In [ ]:
# Display just one
first = id_ra_dec[0]
ra, dec = first['RA'], first['DEC']
cutout = display_cutout_image(butler, ra, dec, label="Object ID: %d" % id_ra_dec[0]['ID'])

Here when we browse around we get both x,y coordinates (RA, Dec) = $(\alpha, \delta)$ the pixel counts as well as the list of named mask bits that are set for the pixels.

# Mask planes

Along with the pixels, the coadd image also has a set of associated "mask" planes. "Mask" means has an identified property -- it doesn't necessarily mean "bad".  In particular, the "DETECTED" mask plane means the measurement identified an object here.  "Plane" here refers to a specific bit in the mask.

Specifically, the blue overlays above are called the "footprints" for the observations.  These are the pixels identified by the pipeline as "belonging" to the object: pixels where the object contributes detectable number of photons above the sky background.

The colors are configurable, but the default values are:

In [ ]:
display = afwDisplay.getDisplay()
for maskName, maskBit in cutout.mask.getMaskPlaneDict().items():
    print('{}: {}'.format(maskName, display.getMaskPlaneColor(maskName)))

### Differences between the `plt.imshow` and `afwDisplay`:
* Color scale inverted  
    Arbitrary choice.  You could choose many different color scales.
* X,Y origin flipped vertically  
    Note that `afwDisplay` shows the pixels with the lower-left as the origin, which is the x,y convention that most  astronomers are using to thinking in.  This is opposite to the vertical orientation in `matplotlib.pyplot.imshow`.  We also get the original pixel coordinates of the tract+patch coadd image, which is potentially useful.
* We've lost our color bar.  
  While in principle, this is worse, in practice, we didn't know what the values meant anyway (counts/sec, nanoJanskys?) and as long as we can see the sky noise we're getting a sense of the significance.
* MWV doesn't know how to save the displayed image as a PNG.

Take a look at https://pipelines.lsst.io/v/DM-11077/getting-started/display.html for a few more details.

### PSF Model

The cutout image object retains full information from the image, including the PSF model.

The PSF model is accessible as a function object that can be evaluated a specific locations.

In addition, the coadd catalog saves the xx, xy, yy moments of the PSF model at the location of photometered objects.

In [ ]:
# We still have our cutout image object from above
psf = cutout.getPsf()

In [ ]:
print(psf)

The repition in the object name is a consequence of the pybind11 mapping.  It's really a `lsst.meas.algorithms.coaddPsf` object.  The C++ documentation unfortunately doesn't come through nicely through the wrapping to Python:

In [ ]:
help(psf)

We can look up the documentation in the LSST DM Stack Doxygen pages.  If we assume (correctly for these purposes) that a CoaddPsf is just like a regular PSF, then we can look here:
http://doxygen.lsst.codes/stack/doxygen/x_masterDoxyDoc/classlsst_1_1afw_1_1detection_1_1_psf.html

where we will see that there are several relevant functions:
`computeImage`: Return an Image of the PSF, in a form that can be compared directly with star images.
`computeKernelImage`:  Return an Image of the PSF, in a form suitable for convolution.
`computeShape`: Compute the ellipse corresponding to the second moments of the Psf.

and then some interesting things you might not have expected to even be available:
`getAverageColor`: Return the average Color of the stars used to construct the Psf.
`getAveragePosition`: Return the average position of the stars used to construct the Psf. 

Unfortunately, the documentation is hard to read from a Python perspective.  What kind of arguments does the following function actually want and how do I create those objects?
```
computeImage(...) from builtins.PyCapsule
        computeImage(self: lsst.afw.detection._psf.Psf, position: lsst.afw.geom.coordinates.coordinates.Point2D=Point2D(nan, nan), color: lsst.afw.image.color.Color=<lsst.afw.image.color.Color object at 0x2b8f57b38928>, owner: lsst.afw.detection._psf.ImageOwnerEnum=ImageOwnerEnum.COPY) -> lsst.afw.image.image.image.ImageD)
```